In [1]:
import pandas as pd
import random
import datetime
import numpy as np
from itertools import repeat
import pickle
import math
import time
import gurobipy as gp
from gurobipy import *

import matplotlib
import matplotlib.pylab as plt


def create_dataframe(excel_data_path):
    """
    Takes in order data in excel format, returns order and equipments in pandas dataframes
    """
    xls = pd.ExcelFile(excel_data_path)
    items = pd.read_excel(xls, 'items')
    bins = pd.read_excel(xls, 'bins')

    return items, bins


filepath = "C:/Users/Ashrafur.Rahman/OneDrive - Blume Global/Data and Codes/Container Loading/data/3d-bin-packing/india-team/3d-bin-packing-toy-large-shared-w-neeraj (3).xlsx"
#filepath="C:/Users/Ashrafur.Rahman/OneDrive - Blume Global/Data and Codes/Container Loading/data/transLoading.xlsx"

items, bins = create_dataframe(filepath)
#items.sort_values(by=['volume', 'length', 'height', 'weight'], ascending=[False, False, False, False], inplace=True)


In [2]:
items

,item,length,width,height,volume,weight,class,stackability,customer,priority,Unnamed: 10,x,y,z,w,d,h,x_dim,y_dim,z_dim
0,1,6,6,5,180,723,A,1,2,1.0,NaN,0,0,0,6,6,5,0,0,0
1,2,6,6,5,180,692,A,1,3,2.0,NaN,0,6,0,6,6,5,0,6,0
2,3,6,6,5,180,727,A,1,1,1.0,NaN,6,0,0,6,6,5,6,0,0
3,4,6,6,5,180,790,A,1,1,1.0,NaN,6,6,0,6,6,5,6,6,0
4,5,6,6,5,180,740,A,1,3,1.0,NaN,12,0,0,6,6,5,12,0,0
5,6,6,6,5,180,835,A,1,2,1.0,NaN,12,6,0,6,6,5,12,6,0
6,7,6,6,5,180,856,A,1,3,2.0,NaN,18,0,0,6,6,5,18,0,0
7,8,6,6,5,180,570,A,1,2,2.0,NaN,18,6,0,6,6,5,18,6,0
8,9,6,6,5,180,933,B,1,3,2.0,NaN,24,0,0,6,6,5,24,0,0
9,10,6,6,5,180,749,B,1,1,1.0,NaN,24,6,0,6,6,5,24,6,0


In [3]:
x=items.x.tolist()
y=items.y.tolist()
z=items.z.tolist()


l=items.w.tolist()
w=items.d.tolist()
h=items.h.tolist()


In [4]:
IP=list(range(len(items)))

In [5]:
L,H,W=40,12,12

In [6]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from itertools import product
%matplotlib inline
%matplotlib nbagg
from matplotlib import cm

fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(111, projection='3d')

# create list of corners
#z = list(product([-1,1], repeat=3))

FLB=(0,0,0)
FLT=(0,0,H)
FRT=(0,W,H)
FRB=(0,W,0)

RLB=(L,0,0)
RLT=(L,0,H)
RRT=(L,W,H)
RRB=(L,W,0)

verts=[  [FLB,FLT,FRT,FRB], [FLB,FLT,RLT,RLB],[FLT,FRT,RRT,RLT],[RRT,FRT,FRB,RRB],[RLB,FLB,FRB,RRB],[RLB,RLT,RRT,RRB]

]
ax.set_xlim3d(0,L)
ax.set_ylim3d(0,L)
ax.set_zlim3d(0,L)
ax.set_xlabel('Length')
ax.set_ylabel('Width')
ax.set_zlabel('Height')
ax.grid(False)
# plot sides
ax.add_collection3d(Poly3DCollection(verts,facecolors='blue', linewidths=0.25, edgecolors='grey', alpha=.025))



# set verts connectors
#verts = [[z[0],z[1],z[5],z[4]], [z[4],z[6],z[7],z[5]], [z[7], z[6], z[2], z[3]], [z[2], z[0], z[1], z[3]],
 #        [z[5], z[7], z[3], z[1]], [z[0], z[2], z[6], z[4]]]

flb= {}
flt= {}
frt={}
frb={}

rlb={}
rlt={}
rrt={}
rrb={}


c=['red','green','blue','yellow','orange']
for i in IP:
    flb[i]=(x[i],y[i],z[i])
    flt[i]=(x[i],y[i],z[i]+h[i])
    frt[i]=(x[i],y[i]+w[i],z[i]+h[i])
    frb[i]=(x[i],y[i]+w[i],z[i])

    rlb[i]=(x[i]+l[i],y[i],z[i])
    rlt[i]=(x[i]+l[i],y[i],z[i]+h[i])
    rrt[i]=(x[i]+l[i],y[i]+w[i],z[i]+h[i])
    rrb[i]=(x[i]+l[i],y[i]+w[i],z[i])

    q=[  [flb[i],flt[i],frt[i],frb[i]],
            [flb[i],flt[i],rlt[i],rlb[i]],
            [flt[i],frt[i],rrt[i],rlt[i]],
            [rrt[i],frt[i],frb[i],rrb[i]],
            [rlb[i],flb[i],frb[i],rrb[i]],
            [rlb[i],rlt[i],rrt[i],rrb[i]]
            ]
    face=i
    color=random.choice(c)
    ax.add_collection3d(Poly3DCollection(q,facecolors=color, linewidths=1, edgecolors='black', alpha=0.99))
    ax.text((x[i]+l[i]),(y[i]+w[i]/2),(z[i]+h[i]/2),   face,'y', size=7, zorder=30,
    color='black')
    ax.text((x[i]+l[i]/2),(y[i]),(z[i]+h[i]/2),   face,'x', size=7, zorder=30,
    color='black')
    ax.text((x[i]+l[i]/2),(y[i]+w[i]/2),(z[i]+h[i]),   face,'x', size=7, zorder=30
            ,
    color='black')

#'(%d, %d, %d), dir=%s' % (x, y, z, zdir)


label1 = "FRONT"
label2 = "REAR"
#label3 = "BOTTOM"
ax.text(0, W/2, H/2, '%s' % (label1), size=10,zorder=7, color='black')
ax.text(L, W/2, H/2, '%s' % (label2), size=10,zorder=10, color='blue')
#ax.text(6, 2, 0, '%s' % (label3), size=20,zorder=5, color='blue')
#ax.view_init(elev=0, azim=45)
plt.show()

<IPython.core.display.Javascript object>

In [7]:
plt.savefig('3-d-loading-oc.png')

In [10]:
flb


{0: (10, 7, 0),
 1: (15, 7, 0),
 2: (20, 7, 0),
 3: (25, 7, 0),
 4: (30, 7, 0),
 5: (35, 7, 0),
 6: (8, 0, 0),
 7: (13, 0, 0),
 8: (18, 0, 0),
 9: (23, 0, 0),
 10: (28, 0, 0),
 11: (33, 0, 0),
 12: (0, 0, 7),
 13: (0, 6, 7),
 14: (6, 6, 7),
 15: (12, 6, 7),
 16: (18, 6, 7),
 17: (22, 6, 7),
 18: (26, 6, 7),
 19: (30, 6, 7),
 20: (34, 6, 7),
 21: (5, 0, 7),
 22: (9, 0, 7),
 23: (33, 4, 0),
 24: (36, 4, 0),
 25: (28, 4, 0),
 26: (0, 0, 0),
 27: (0, 7, 0),
 28: (5, 7, 0)}

In [8]:
items

,item,length,width,height,volume,weight,class,stackability,customer,priority,Unnamed: 10,x,y,z,w,d,h,x_dim,y_dim,z_dim
0,1,5,6,4,120,723,A,1,2,1,NaN,10,7,0,5,4,6,15,11,6
1,2,5,6,4,120,692,A,1,3,2,NaN,15,7,0,5,4,6,20,11,6
2,3,5,6,4,120,727,A,1,1,1,NaN,20,7,0,5,4,6,25,11,6
3,4,5,6,4,120,790,A,1,1,1,NaN,25,7,0,5,4,6,30,11,6
4,5,5,6,4,120,740,A,1,3,1,NaN,30,7,0,5,4,6,35,11,6
5,6,5,6,4,120,835,A,1,2,1,NaN,35,7,0,5,4,6,40,11,6
6,7,5,6,4,120,856,A,1,3,2,NaN,8,0,0,5,4,6,13,4,6
7,8,5,6,4,120,570,A,1,2,2,NaN,13,0,0,5,4,6,18,4,6
8,9,5,4,6,120,933,B,1,3,2,NaN,18,0,0,5,4,6,23,4,6
9,10,5,4,6,120,749,B,1,1,1,NaN,23,0,0,5,4,6,28,4,6


In [20]:
p=zone_ratio*5

In [22]:
[x*5 for x in zone_ratio]

[5, 0, 2.5, 2.5]

In [24]:
L=40

In [25]:
if  p[0]>=L/4 and p[0]<L/2  and p[0]+l[i]> L/2  and p[0]+l[i]<=3*L/4:
    print('6th weight condition')
    zone_ratio=[0,(L/2-p[0])/l[i],(p[0]+l[i]-l/2)/l[i],0]

In [27]:
import numpy as np

In [28]:
a = np.random.randint(0,20,10)

In [29]:
a

array([19, 15,  0,  9, 14, 11, 10, 13,  4,  0])

In [30]:
ix = np.arange(a.size)

In [31]:
ix

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [32]:
will_swap = np.random.random(a.size) <= 0.2

In [34]:
after_swap = np.random.permutation(ix[will_swap])

In [35]:
ix[will_swap] = after_swap # update ix with the swapped candidates
a = a[ix]
print(a)

[19 15  0  9 14 11 10 13  4  0]
